# [LangChain Crash Course - Build apps with language models](https://www.youtube.com/watch?v=LbT1yp6quS8)

## Overview
* Installation
* LLMs
* Prompt Templates
* Chains
* Agents and Tools
* Memory
* Document Loaders
* Indexes

## Installation

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
%load_ext dotenv
%dotenv

## 1. LLMs

A generic interface for all LLMS. See all LLM providers: [https://python.langchain.com/en/latest/modules/models/llms/integrations.html](https://python.langchain.com/en/latest/modules/models/llms/integrations.html)


In [9]:
import os

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9) # model_name="text-davinci-003"
text = "What is a good company name for a company that sells colorful socks?"
print(llm(text))



Happy Socksy.


In [10]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

from langchain import HuggingFaceHub

# https://huggingface.co/google/flan-t5-xl
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature": 0, "max_length": 64})

llm("Translate English to Spanish: How old are you?")

ValueError: Error raised by inference API: Internal Server Error